In [1]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [2]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

In [10]:
df = pd.read_pickle('askreddit_lemmatized.pickle')
df2 = df[0].tolist()
df2

['victorias secret originally suppose place man buy lingerie wife feel embarrassed exclusively shop woman',
 'ford mustang   suppose car big trunk fit grocery   ad article state mustang match color lipstick   make sense lot old mustang red',
 'heel remember correctly',
 'family feud get new question answer',
 'skirt wear battle man room',
 'ugg boot originally wear male surfer girlfriend steal wear like leather jacket',
 'filter cigarette general marlboro big leap term gendere marketing conspiracy theory come like cigarette idk bad people switch filter cigarette filter mean good time salt earth classic american marlboro man come existence try distance brand previously woman focus advertisement',
 'leg stirrup use gynocologist invent remove kidney stone man cut testicle cut right prostate low survival rate approve medical profession    original hippocratic oath section cut stone meaning swear surgically remove kidney stone later remove section oath bunch urologist job',
 'hoodie apparen

In [3]:
data_df = pd.read_pickle('cleaned_list_df.pickle')
data_df = data_df.T
data_df

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
0,in the future when apple is at a share your g...,the children were not motivated by achieving ...,tldr do whatever the fuck you want,victorias secret was originally supposed to be...,so was there any posts left after the purge,but i wanted the robot ferret,meme of the month seeding and nominations for ...,redditmc is opening staff positions builders h...,it looks like there may be more to this story ...,decisions regarding mental health care canada...,wait hold the fucking phone he spent like mi...,hi there i ve been looking for a stable job f...,holy mackerel you guys the spotlight is for wr...
1,you ever stare at your account and think wow h...,also proved children like to gamble according ...,so youre telling me people on the internet lie,the ford mustang it was supposed to be a car ...,ok just for some context facebook has an esti...,haha suck it you fools i already got my roboti...,dont have any examples and im not sure if it c...,using this to see if i have new flair,some series get old after a couple updates thi...,public perceptions of exonerees through crime...,trump on harris nasty woman trump on clinton ...,hello people i hope yall are doing good i was ...,congrats ucalamityjeans as tens said i hope ...
2,buy at the top gang 🤡,kind of like how gamers go through rpg games e...,,heels if i remember correctly,they still havent taken down my post about mar...,this not funny rule number oh i forget carry on,certified hood classic,cries in bedrock edition,omg man with the skull cap is the mvp this tim...,does child engagement in household tasks pred...,i could listen to kamala all day every day spe...,company apple i am a bot and this action was...,congratulations it looks like youve been overl...
3,almost responded to an email with thanks retar...,i teach elementary pe to year olds one of th...,this is so funny this subreddit got fucked by ...,is this how family feud gets new questions and...,why does facebook have a fake news problem be...,but the nigerian guy is legit right,arnold schwarzerunner godtier,thank you thats very cash money of you,this update otherwise known as ive had it with...,study on attitudes toward police law enforce...,petty fucking manchild cant stand biden and ...,company netflix i am a bot and this action w...,oh yay ive seen you around on prompts and enj...
4,top trending tickers sentiments and options on...,wouldnt this make lootmystery boxes even worse...,a note about celebrity request bans when riama...,skirts used to be worn in battle by men for mo...,a lot of karens are going to be be saying they...,bruh when i was younger i went to a farm this ...,el dorado explanation,it wont let me connect to the server im using ...,does this mean you owe the man with the skull ...,the relationship between peer support family...,who owns antifacom well it appears it was regi...,company other i am a bot and this action was...,congrats ucalamityjeans as is tradition do ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101136,none,none,none,maybe,none,none,none,none,none,none,none,none,none
101137,none,none,none,is it my cake day i have always wondered how p...,none,none,none,none,none,none,none,none,none
101138,none,none,none,theres a cake emoji next to your username on y...,none,none,none,none,none,none,none,none,none
101139,none,none,none,i think my problem is that im using narwhal ed...,none,none,none,none,none,none,none,none,none


In [11]:
df

,0
0,victorias secret originally suppose place man ...
1,ford mustang suppose car big trunk fit groce...
2,heel remember correctly
3,family feud get new question answer
4,skirt wear battle man room
...,...
101136,maybe
101137,cake day wonder people know
101138,s cake emoji username comment cake day check p...
101139,think problem be narwhal edit thank sorry get ...


In [4]:
wsb = data_df['wsb'].tolist()
science = data_df['science'].tolist()
ama = data_df['ama'].tolist()
askreddit = data_df['askreddit'].tolist()
worldnews = data_df['worldnews'].tolist()
funny = data_df['funny'].tolist()
dankmemes = data_df['dankmemes'].tolist()
memes = data_df['memes'].tolist()
nosleep = data_df['nosleep'].tolist()
psychology = data_df['psychology'].tolist()
politics = data_df['politics'].tolist()
cscareerquestions = data_df['cscareerquestions'].tolist()
writingprompts = data_df['writingprompts'].tolist()

full_list = wsb + science + ama + askreddit + worldnews + funny + dankmemes + memes + nosleep + psychology + politics + cscareerquestions + writingprompts
full_list

['in the future when apple is at  a share your gran kids will ask why you didnt buy apple at  and youll answer with i bought corn instead',
 'you ever stare at your account and think wow how did i put  into options and think that was safe',
 'buy  at the top gang 🤡',
 'almost responded to an email with thanks retard because  of my social interaction is now through this sub',
 'top trending tickers sentiments and options on rwallstreetbets for the day of august    ────────────────  trending tickers  tickercompanymentionsbulls  bearspricechangepe  tslatesla    spyexchange traded    slvexchange traded    aaplapple    cornexchange traded    rktrocket companies    msftmicrosoft    amdadvanced micro    bathe boeing    fbfacebook    prplpurple innovation    qqqexchange traded    netcloudflare    cscocisco systems    nvdanvidia      ────────────────  trending options  tickerstriketypeexpiration                   ────────────────  trending sentiment  sentimentcomments       ────────────────  ot

In [12]:
vectorizer = TfidfVectorizer(binary=True)

doc_word = vectorizer.fit_transform(df[0])
words = list(np.asarray(vectorizer.get_feature_names())) 
doc_word

<101141x46045 sparse matrix of type '<class 'numpy.float64'>'
	with 995025 stored elements in Compressed Sparse Row format>

In [13]:
doc_word

<101141x46045 sparse matrix of type '<class 'numpy.float64'>'
	with 995025 stored elements in Compressed Sparse Row format>

In [14]:
# cvn = TfidfVectorizer(ngram_range=(1,1))
# data_cvn = cvn.fit_transform(full_list)
# data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# # data_dtmn.columns = data_df.index
# data_dtmn

In [15]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(doc_word))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in vectorizer.vocabulary_.items())
word2id = dict((k, v) for k, v in vectorizer.vocabulary_.items())

In [17]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=20, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 20:07:07: using symmetric alpha at 0.05
INFO - 20:07:07: using symmetric eta at 0.05
INFO - 20:07:07: using serial LDA version on this node
INFO - 20:07:07: running online (multi-pass) LDA training, 20 topics, 50 passes over the supplied corpus of 46045 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 20:07:07: PROGRESS: pass 0, at document #2000/46045


IndexError: index 50527 is out of bounds for axis 1 with size 46045

In [8]:
ldan.save('askreddit_bigrams.model')

INFO - 19:29:04: saving LdaState object under askreddit_bigrams.model.state, separately None
INFO - 19:29:04: storing np array 'sstats' to askreddit_bigrams.model.state.sstats.npy
INFO - 19:29:04: saved askreddit_bigrams.model.state
INFO - 19:29:04: saving LdaMulticore object under askreddit_bigrams.model, separately ['expElogbeta', 'sstats']
INFO - 19:29:04: storing np array 'expElogbeta' to askreddit_bigrams.model.expElogbeta.npy
INFO - 19:29:04: not storing attribute id2word
INFO - 19:29:04: not storing attribute dispatcher
INFO - 19:29:04: not storing attribute state
INFO - 19:29:04: saved askreddit_bigrams.model


In [9]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [10]:
from gensim import corpora
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

## 20 topics, tdfidf, askreddit

In [11]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 19:44:28: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 19:44:28: NumExpr defaulting to 8 threads.


## 10 topics, TF-IDF'ed

In [27]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

## 10 topics, CountVectorized

In [20]:
vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 14:13:40: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 14:13:40: NumExpr defaulting to 8 threads.
